In [ ]:
! cat README.md

In [ ]:
# parse data

with open('input.txt', 'rt') as f:
    lines = f.read().splitlines()
fields, myticket, nearby = {}, None, []
part = 'fields'
for l in lines:
    if len(l) > 0:
        if part == 'fields':
            k, v = tuple(l.split(':'))
            v = list(map(lambda x: tuple(map(int, x.split('-'))), v.split(' or ')))
            fields[k] = v
        if part == 'nearby':
            nearby.append(list(map(int, l.split(','))))
        if part == 'your':
            myticket = list(map(int, l.split(',')))
        if l.startswith('nearby'):
            part = 'nearby'
        if l.startswith('your'):
            part = 'your'
    else:
        part = None

In [ ]:
def invalid_value(ticket, fields):
    
    acc = 0
    for n in ticket:
        invalid = False
        for k, v in fields.items():
            if (v[0][0] <= n <= v[0][1]) or (v[1][0] <= n <= v[1][1]):
                invalid = False
                break
            else:
                invalid = True
        if invalid:
            acc += n
    return acc

In [ ]:
# test
test_fields = {'class': [(1,3), (5,7)], 
               'row': [(6,11), (33,44)],
               'seat': [(13,40), (45,50)]}
test_nearby = [[7,3,47],[40,4,50],[55,2,20],[38,6,12]]
sum([invalid_value(t, test_fields) for t in test_nearby])  

In [ ]:
# solution
sum([invalid_value(t, fields) for t in nearby])

In [ ]:
def is_invalid(ticket):
    
    acc = 0
    for n in ticket:
        invalid = False
        for k, v in fields.items():
            if (v[0][0] <= n <= v[0][1]) or (v[1][0] <= n <= v[1][1]):
                invalid = False
                break
            else:
                invalid = True
        if invalid:
            return invalid
    return invalid

In [ ]:
nearby_valid = [t for t in nearby if not is_invalid(t)]

In [ ]:
import numpy as np

def is_valid_field(k, fields, values):
    
    v = fields[k]
    return np.all([(v[0][0] <= n <= v[0][1]) or (v[1][0] <= n <= v[1][1]) for n in values])

def get_matches(fields, tickets):
    
    match_dict = {}
    values = list(zip(*tickets))
    for k in fields:
        for i, v in enumerate(values):
            if is_valid_field(k, fields, v):
                match_dict[k] = match_dict.get(k, set()) | set([i])
    return match_dict

In [ ]:
# search

def is_solution(d):
    return np.all([len(v)==1 for k, v in d.items()])

def is_feasible(d):
    return np.all([len(v) >= 1 for k, v in d.items()])

def children(d, k):
    for v in d[k]:
        r = d
        r[k] = set([v])
        for w in d:
            if w != k:
                r[w] = r[w] - set([v])
        yield r

In [ ]:
def search(fields, tickets):
    d = get_matches(fields, tickets)
    queue = [d]
    while not is_solution(d):
        for k in sorted(d, key=lambda x:len(d[x])):
            for child in children(d, k):
                if is_solution(child):
                    return child
                elif is_feasible(child):
                    queue.append(child)
        d = queue.pop()

In [ ]:
# test
test_fields = {'class': [(0,1), (4,19)], 
               'row': [(0,5), (8,19)],
               'seat': [(0,13), (6,19)]}
test_nearby = [[3,9,18],[15,1,5],[5,14,9]]

# your ticket: 11,12,13
search(test_fields, test_nearby)

In [ ]:
search(fields, nearby_valid)

In [ ]:
# solution
from functools import reduce
assignment = search(fields, nearby_valid)
reduce(lambda x,y:x*y, [myticket[list(assignment[k])[0]] for k in fields if k.startswith('departure')])